In [44]:
!pip install pandas matplotlib
!pip install -U kaleido


In [58]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import numpy as np

def visualize_gene_chemical_interaction(file_path, output_path):
    # Load the data
    data = pd.read_csv(file_path)

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges to the graph
    for index, row in data.iterrows():
        gene = row['Genes']
        chemical = row['ChemicalName']
        interaction_type = row['Interaction type']
        regulation = row['Regulation']

        if pd.notna(gene) and pd.notna(chemical):
            G.add_edge(gene, chemical, interaction=f"{interaction_type}, {regulation}")

    # Create a layout for nodes
    pos = nx.spring_layout(G, k=0.5, seed=42)  # Using spring layout

    # Define edge colors based on interaction type
    edge_colors = {'positive': '#1f77b4', 'negative': '#ff7f0e', 'neutral': '#2ca02c'}

    # Create edge trace
    edge_trace = []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        interaction_details = edge[2]['interaction']
        interaction_type = interaction_details.split(',')[0].lower()
        edge_trace.append(
            go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                line=dict(width=2, color=edge_colors.get(interaction_type, '#333')),
                hoverinfo='text',
                text=[interaction_details],
                mode='lines'
            )
        )

    # Create node trace
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers+text',
        textposition='top center',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='Viridis',
            color=[],
            size=10,  # Fixed size for small circles
            colorbar=dict(
                thickness=15,
                title='Node Degree',
                xanchor='left',
                titleside='right'
            ),
        )
    )

    for node in G.nodes():
        x, y = pos[node]
        degree = len(G[node])
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['text'] += tuple([f"{node}<br>{degree}"])  # Show degree without the label
        node_trace['marker']['color'] += tuple([degree])
        node_trace['marker']['size'] = [10] * len(G.nodes())  # Set fixed size for all nodes

    # Create the figure
    fig = go.Figure(
        data=edge_trace + [node_trace],
        layout=go.Layout(
            title='Gene-Chemical Interaction Graph',
            titlefont_size=20,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=20, r=20, t=40),
            xaxis=dict(showgrid=False, zeroline=False),
            yaxis=dict(showgrid=False, zeroline=False),
            updatemenus=[
                {
                    'buttons': [
                        {
                            'args': [None, {'frame': {'duration': 50, 'redraw': True}, 'fromcurrent': True}],
                            'label': 'Play',
                            'method': 'animate'
                        },
                        {
                            'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                            'label': 'Pause',
                            'method': 'animate'
                        }
                    ],
                    'direction': 'left',
                    'pad': {'r': 10, 't': 87},
                    'showactive': True,
                    'type': 'dropdown',
                    'x': 0.1,
                    'xanchor': 'right',
                    'y': 0,
                    'yanchor': 'top'
                }
            ],
            sliders=[{
                'steps': [{
                    'args': [[str(angle)], {'frame': {'duration': 50, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300, 'easing': 'cubic-in-out'}}],
                    'label': str(angle),
                    'method': 'animate'
                } for angle in np.linspace(0, 360, num=30)],
                'currentvalue': {
                    'prefix': 'Angle:',
                    'visible': True,
                    'xanchor': 'right'
                },
                'transition': {'duration': 300, 'easing': 'cubic-in-out'}
            }]
        ),
    )

    # Save the figure as an HTML file
    fig.write_html(output_path)

    # Show the figure
    fig.show()

# Replace 'path_to_your_file.csv' with your actual file path
file_path = '/content/Task10_Gene_to_Chemical.csv'
output_path = '/content/gene_chemical_interaction_graph.html'
visualize_gene_chemical_interaction(file_path, output_path)
